Weather data that is needed

In [19]:
import pandas as pd
import json
import pandas as pd
from mechanize import Browser
from bs4 import BeautifulSoup
import numpy as np
from collections import OrderedDict
import pickle
from __future__ import print_function

In [20]:
base_url = 'https://www.ncdc.noaa.gov/cag/county/time-series/'
avg_temp_url = "-tavg-1-1-1984-2018.json?base_prd=true&begbaseyear=1984&endbaseyear=2018"
max_temp_url = "-tmax-1-1-1984-2018.json?base_prd=true&begbaseyear=1984&endbaseyear=2018"
min_temp_url = "-tmin-1-1-1984-2018.json?base_prd=true&begbaseyear=1984&endbaseyear=2018"
pcp_url = "-pcp-1-1-1984-2018.json?base_prd=true&begbaseyear=1984&endbaseyear=2018"

url_list = OrderedDict(sorted({'avg_temp':avg_temp_url, 'max_temp': max_temp_url, 'min_temp': min_temp_url, 'percipitation':pcp_url}.items()))

states = OrderedDict(sorted({'Alabama': 'AL', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT',\
          'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana':\
          'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', \
          'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT'\
          , 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY'\
          , 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': \
          'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', \
          'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', \
          'Wyoming': 'WY'}.items()))

def read_data(url,col, state_name):   
    browser = Browser()
    browser.open(url)
    r_json=json.loads(browser.response().read())
    response_df = pd.DataFrame(r_json['data']).transpose()
    if (not response_df.empty) and ('value' in response_df.columns):
        response_df = response_df.drop(columns='anomaly').reset_index()
        response_df.loc[:,'County']=r_json['description']['title'].split(',')[0].replace(' County','')
        response_df.loc[:,'Year']=response_df['index'].str[:4]
        response_df.loc[:,'State']=state_name
        response_df = response_df.rename(columns={'value':col}).drop(columns='index')
    else:
        county_name = r_json['description']['title'].split(',')[0].replace(' County','')
        response_df=pd.DataFrame({'Year':np.nan,'State':state_name,'County':county_name,'avg_temp':np.nan},index=[0])
    return response_df.set_index(['Year','State','County'])

def get_county_codes(state_code):
    br = Browser()
    br.open("https://www.ncdc.noaa.gov/cag/county/time-series/"+state_code+"-003/tavg/1/1/1984-2018?base_prd=true&firstbaseyear=1985&lastbaseyear=2000")
    bsoup = BeautifulSoup(br.response().read())
    sub = bsoup.find(id="div")
    counties=[]
    for a in sub:
        if 'value' in str(a):
            counties.append(str(a).split('value=')[1][1:7])
    return counties

In [21]:
state_counties_code= OrderedDict()
try:
    with open('./Data/pickle/state_counties_code.pickle', 'rb') as handle:
        state_counties_code = pickle.load(handle)
except:
    for state, code in states.iteritems():
        print(code)
        state_counties_code[state] = get_county_codes(code)
    with open('./Data/pickle/state_counties_code.pickle', 'wb') as handle:
        pickle.dump(state_counties_code, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(state_counties_code)

OrderedDict([('Alabama', ['AL-001', 'AL-003', 'AL-005', 'AL-007', 'AL-009', 'AL-011', 'AL-013', 'AL-015', 'AL-017', 'AL-019', 'AL-021', 'AL-023', 'AL-025', 'AL-027', 'AL-029', 'AL-031', 'AL-033', 'AL-035', 'AL-037', 'AL-039', 'AL-041', 'AL-043', 'AL-045', 'AL-047', 'AL-049', 'AL-051', 'AL-053', 'AL-055', 'AL-057', 'AL-059', 'AL-061', 'AL-063', 'AL-065', 'AL-067', 'AL-069', 'AL-071', 'AL-073', 'AL-075', 'AL-077', 'AL-079', 'AL-081', 'AL-083', 'AL-085', 'AL-087', 'AL-089', 'AL-091', 'AL-093', 'AL-095', 'AL-097', 'AL-099', 'AL-101', 'AL-103', 'AL-105', 'AL-107', 'AL-109', 'AL-111', 'AL-113', 'AL-115', 'AL-117', 'AL-119', 'AL-121', 'AL-123', 'AL-125', 'AL-127', 'AL-129', 'AL-131', 'AL-133']), ('Arizona', ['AZ-001', 'AZ-003', 'AZ-005', 'AZ-007', 'AZ-009', 'AZ-011', 'AZ-012', 'AZ-013', 'AZ-015', 'AZ-017', 'AZ-019', 'AZ-021', 'AZ-023', 'AZ-025', 'AZ-027']), ('Arkansas', ['AR-001', 'AR-003', 'AR-005', 'AR-007', 'AR-009', 'AR-011', 'AR-013', 'AR-015', 'AR-017', 'AR-019', 'AR-021', 'AR-023', 'AR

In [34]:
try:
    data_df = pd.read_pickle("./Data/pickle/data_df.pkl")
except:
    data_df = pd.DataFrame()

for col, url in url_list.iteritems():
    print(col)
    
    if col in data_df.columns:
        print('Found col')
        if sorted(list(data_df.index.levels[1]))==sorted(state_counties_code.keys()):
            print('Found saved data for {c}, skipping data point.'.format(c=col))
            continue
        else:
            try:
                print('reading pickle')
                inner_df = pd.read_pickle("./Data/pickle/inner_df.pkl")
            except:
                print('exception empty inner_df')
                inner_df = pd.DataFrame()
    else:
        print('Col not found in data_df, looking into inner_df')
        try:
            print('reading pickle')
            inner_df = pd.read_pickle("./Data/pickle/inner_df.pkl")
        except:
            print('exception empty inner_df')
            inner_df = pd.DataFrame()
        if col in inner_df.columns:
            print('col found in inner_df')
            pass
        else:
            print('new col for inner_df loading it empty.')
            inner_df = pd.DataFrame()

    
    for state, counties in state_counties_code.iteritems():
        print('\t',state)
        
        if (not inner_df.empty):
            if state in inner_df.index.levels[1]:
                print('\t Found saved data for {s}, skipping it.'.format(s=state))
                continue
        else:
            print('\t new state')
            pass
        
        for county in counties:
            print('\t\t',county)
            if inner_df.empty:
                print('\t\t overwritting inner df')
                inner_df = read_data(base_url+county+url,col, state)
            else:
                print('\t\t appending to inner df')
                inner_df = pd.concat([inner_df,read_data(base_url+str(county)+url,col, state)],axis=0)
        inner_df.to_pickle("./Data/pickle/inner_df.pkl")

        
    if data_df.empty:
        data_df = inner_df.copy()
    else:
        data_df = data_df.merge(inner_df,how='outer',left_index=True,right_index=True)
    data_df.to_pickle("./Data/pickle/data_df.pkl")

data_df.head()

avg_temp
Found col
Found saved data for avg_temp, skipping data point.
max_temp
Found col
Found saved data for max_temp, skipping data point.
min_temp
Found col
Found saved data for min_temp, skipping data point.
percipitation
Col not found in data_df, looking into inner_df
reading pickle
col found in inner_df
	 Alabama
	 Found saved data for Alabama, skipping it.
	 Arizona
	 Found saved data for Arizona, skipping it.
	 Arkansas
	 Found saved data for Arkansas, skipping it.
	 California
	 Found saved data for California, skipping it.
	 Colorado
	 Found saved data for Colorado, skipping it.
	 Connecticut
	 Found saved data for Connecticut, skipping it.
	 Delaware
	 Found saved data for Delaware, skipping it.
	 Florida
	 Found saved data for Florida, skipping it.
	 Georgia
	 Found saved data for Georgia, skipping it.
	 Idaho
	 Found saved data for Idaho, skipping it.
	 Illinois
	 Found saved data for Illinois, skipping it.
	 Indiana
	 Found saved data for Indiana, skipping it.
	 Iowa
	 F

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:55: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



		 NC-033
		 appending to inner df
		 NC-035
		 appending to inner df
		 NC-037
		 appending to inner df
		 NC-039
		 appending to inner df
		 NC-041
		 appending to inner df
		 NC-043
		 appending to inner df
		 NC-045
		 appending to inner df
		 NC-047
		 appending to inner df
		 NC-049
		 appending to inner df
		 NC-051
		 appending to inner df
		 NC-053
		 appending to inner df
		 NC-055
		 appending to inner df
		 NC-057
		 appending to inner df
		 NC-059
		 appending to inner df
		 NC-061
		 appending to inner df
		 NC-063
		 appending to inner df
		 NC-065
		 appending to inner df
		 NC-067
		 appending to inner df
		 NC-069
		 appending to inner df
		 NC-071
		 appending to inner df
		 NC-073
		 appending to inner df
		 NC-075
		 appending to inner df
		 NC-077
		 appending to inner df
		 NC-079
		 appending to inner df
		 NC-081
		 appending to inner df
		 NC-083
		 appending to inner df
		 NC-085
		 appending to inner df
		 NC-087
		 appending to inner df
		 NC-089
		 appendi

		 OK-019
		 appending to inner df
		 OK-021
		 appending to inner df
		 OK-023
		 appending to inner df
		 OK-025
		 appending to inner df
		 OK-027
		 appending to inner df
		 OK-029
		 appending to inner df
		 OK-031
		 appending to inner df
		 OK-033
		 appending to inner df
		 OK-035
		 appending to inner df
		 OK-037
		 appending to inner df
		 OK-039
		 appending to inner df
		 OK-041
		 appending to inner df
		 OK-043
		 appending to inner df
		 OK-045
		 appending to inner df
		 OK-047
		 appending to inner df
		 OK-049
		 appending to inner df
		 OK-051
		 appending to inner df
		 OK-053
		 appending to inner df
		 OK-055
		 appending to inner df
		 OK-057
		 appending to inner df
		 OK-059
		 appending to inner df
		 OK-061
		 appending to inner df
		 OK-063
		 appending to inner df
		 OK-065
		 appending to inner df
		 OK-067
		 appending to inner df
		 OK-069
		 appending to inner df
		 OK-071
		 appending to inner df
		 OK-073
		 appending to inner df
		 OK-075
		 appendi

		 SD-025
		 appending to inner df
		 SD-027
		 appending to inner df
		 SD-029
		 appending to inner df
		 SD-031
		 appending to inner df
		 SD-033
		 appending to inner df
		 SD-035
		 appending to inner df
		 SD-037
		 appending to inner df
		 SD-039
		 appending to inner df
		 SD-041
		 appending to inner df
		 SD-043
		 appending to inner df
		 SD-045
		 appending to inner df
		 SD-047
		 appending to inner df
		 SD-049
		 appending to inner df
		 SD-051
		 appending to inner df
		 SD-053
		 appending to inner df
		 SD-055
		 appending to inner df
		 SD-057
		 appending to inner df
		 SD-059
		 appending to inner df
		 SD-061
		 appending to inner df
		 SD-063
		 appending to inner df
		 SD-065
		 appending to inner df
		 SD-067
		 appending to inner df
		 SD-069
		 appending to inner df
		 SD-071
		 appending to inner df
		 SD-073
		 appending to inner df
		 SD-075
		 appending to inner df
		 SD-077
		 appending to inner df
		 SD-079
		 appending to inner df
		 SD-081
		 appendi

		 TX-169
		 appending to inner df
		 TX-171
		 appending to inner df
		 TX-173
		 appending to inner df
		 TX-175
		 appending to inner df
		 TX-177
		 appending to inner df
		 TX-179
		 appending to inner df
		 TX-181
		 appending to inner df
		 TX-183
		 appending to inner df
		 TX-185
		 appending to inner df
		 TX-187
		 appending to inner df
		 TX-189
		 appending to inner df
		 TX-191
		 appending to inner df
		 TX-193
		 appending to inner df
		 TX-195
		 appending to inner df
		 TX-197
		 appending to inner df
		 TX-199
		 appending to inner df
		 TX-201
		 appending to inner df
		 TX-203
		 appending to inner df
		 TX-205
		 appending to inner df
		 TX-207
		 appending to inner df
		 TX-209
		 appending to inner df
		 TX-211
		 appending to inner df
		 TX-213
		 appending to inner df
		 TX-215
		 appending to inner df
		 TX-217
		 appending to inner df
		 TX-219
		 appending to inner df
		 TX-221
		 appending to inner df
		 TX-223
		 appending to inner df
		 TX-225
		 appendi

		 VA-043
		 appending to inner df
		 VA-045
		 appending to inner df
		 VA-047
		 appending to inner df
		 VA-049
		 appending to inner df
		 VA-051
		 appending to inner df
		 VA-053
		 appending to inner df
		 VA-057
		 appending to inner df
		 VA-059
		 appending to inner df
		 VA-061
		 appending to inner df
		 VA-063
		 appending to inner df
		 VA-065
		 appending to inner df
		 VA-067
		 appending to inner df
		 VA-069
		 appending to inner df
		 VA-071
		 appending to inner df
		 VA-073
		 appending to inner df
		 VA-075
		 appending to inner df
		 VA-077
		 appending to inner df
		 VA-079
		 appending to inner df
		 VA-081
		 appending to inner df
		 VA-083
		 appending to inner df
		 VA-085
		 appending to inner df
		 VA-087
		 appending to inner df
		 VA-089
		 appending to inner df
		 VA-091
		 appending to inner df
		 VA-093
		 appending to inner df
		 VA-095
		 appending to inner df
		 VA-097
		 appending to inner df
		 VA-099
		 appending to inner df
		 VA-101
		 appendi

		 WI-055
		 appending to inner df
		 WI-057
		 appending to inner df
		 WI-059
		 appending to inner df
		 WI-061
		 appending to inner df
		 WI-063
		 appending to inner df
		 WI-065
		 appending to inner df
		 WI-067
		 appending to inner df
		 WI-069
		 appending to inner df
		 WI-071
		 appending to inner df
		 WI-073
		 appending to inner df
		 WI-075
		 appending to inner df
		 WI-077
		 appending to inner df
		 WI-078
		 appending to inner df
		 WI-079
		 appending to inner df
		 WI-081
		 appending to inner df
		 WI-083
		 appending to inner df
		 WI-085
		 appending to inner df
		 WI-087
		 appending to inner df
		 WI-089
		 appending to inner df
		 WI-091
		 appending to inner df
		 WI-093
		 appending to inner df
		 WI-095
		 appending to inner df
		 WI-097
		 appending to inner df
		 WI-099
		 appending to inner df
		 WI-101
		 appending to inner df
		 WI-103
		 appending to inner df
		 WI-105
		 appending to inner df
		 WI-107
		 appending to inner df
		 WI-109
		 appendi

avg_temp_x  avg_temp_y max_temp  avg_temp_x min_temp  \
Year State    County                                                            
NaN  Colorado Pitkin            NaN         NaN      NaN         NaN      NaN   
     Florida  St. Lucie         NaN         NaN      NaN         NaN      NaN   
     Georgia  Douglas           NaN         NaN      NaN         NaN      NaN   
              Early             NaN         NaN      NaN         NaN      NaN   
              Echols            NaN         NaN      NaN         NaN      NaN   

                         avg_temp_y percipitation  
Year State    County                               
NaN  Colorado Pitkin            NaN           NaN  
     Florida  St. Lucie         NaN           NaN  
     Georgia  Douglas           NaN           NaN  
              Early             NaN           NaN  
              Echols            NaN           NaN

In [106]:
#Looks like there could some empty columns
print('Columns with all NULLs:\n',data_df.isnull().all())
data_df_copy = data_df.copy()

#Dropping all empty columns
data_df_copy.columns=['avg_temp', 'drop_col1', 'max_temp', 'drop_col2', 'min_temp','drop_col3', 'percipitation']
data_df_copy = data_df_copy.drop(['drop_col1', 'drop_col2', 'drop_col3'],axis=1)
data_df_copy.head()

Columns with all NULLs:
 avg_temp_x       False
avg_temp_y        True
max_temp         False
avg_temp_x        True
min_temp         False
avg_temp_y        True
percipitation    False
dtype: bool


avg_temp max_temp min_temp percipitation
Year State    County                                            
NaN  Colorado Pitkin         NaN      NaN      NaN           NaN
     Florida  St. Lucie      NaN      NaN      NaN           NaN
     Georgia  Douglas        NaN      NaN      NaN           NaN
              Early          NaN      NaN      NaN           NaN
              Echols         NaN      NaN      NaN           NaN

In [109]:
# Dropping rows with no Year value.
print(data_df_copy.shape)
df_to_write = data_df_copy.reset_index()
print(df_to_write[df_to_write['Year'].isnull()].shape)
df_to_write = df_to_write.dropna(how='any',subset=['Year'])
print(df_to_write.shape)
print(df_to_write.head())

(110185, 4)
(1475, 7)
(108710, 7)
      Year    State   County avg_temp max_temp min_temp percipitation
1475  1984  Alabama  Autauga     41.3     53.4     29.3          4.32
1476  1984  Alabama  Baldwin     46.5     56.8     36.1          4.82
1477  1984  Alabama  Barbour     44.0     55.1     32.9          4.29
1478  1984  Alabama     Bibb     39.2     51.0     27.4          3.65
1479  1984  Alabama   Blount     36.6     47.7     25.6          3.61


In [110]:
writer = pd.ExcelWriter('Data/Historical_US_County_Yearly_Weather.xlsx')
df_to_write.to_excel(writer,'CountyWeather',index=False)
writer.save()